<a href="https://colab.research.google.com/github/Data-alchemist-mani/DS-Unit-2-Applied-Modeling/blob/master/module2-wrangle-ml-datasets/Copy_of_LS_DS_232_assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Lambda School Data Science

*Unit 2, Sprint 3, Module 1*

---


# Wrangle ML datasets

- [ ] Continue to clean and explore your data. 
- [ ] For the evaluation metric you chose, what score would you get just by guessing?
- [ ] Can you make a fast, first model that beats guessing?

**We recommend that you use your portfolio project dataset for all assignments this sprint.**

**But if you aren't ready yet, or you want more practice, then use the New York City property sales dataset for today's assignment.** Follow the instructions below, to just keep a subset for the Tribeca neighborhood, and remove outliers or dirty data. [Here's a video walkthrough](https://youtu.be/pPWFw8UtBVg?t=584) you can refer to if you get stuck or want hints!

- Data Source: [NYC OpenData: NYC Citywide Rolling Calendar Sales](https://data.cityofnewyork.us/dataset/NYC-Citywide-Rolling-Calendar-Sales/usep-8jbt)
- Glossary: [NYC Department of Finance: Rolling Sales Data](https://www1.nyc.gov/site/finance/taxes/property-rolling-sales-data.page)

In [0]:
%%capture
import sys

# If you're on Colab:
if 'google.colab' in sys.modules:
    DATA_PATH = 'https://raw.githubusercontent.com/LambdaSchool/DS-Unit-2-Applied-Modeling/master/data/'
    !pip install category_encoders==2.*
    !pip install pandas-profiling==2.*

# If you're working locally:
else:
    DATA_PATH = '../data/'

In [0]:
# Read New York City property sales data
import pandas as pd
df = pd.read_csv(DATA_PATH+'condos/NYC_Citywide_Rolling_Calendar_Sales.csv')

In [0]:
# file is on my computer
from google.colab import files
import pandas as pd
import numpy as np
import seaborn as sns
import category_encoders as ce
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor, DecisionTreeClassifier
from sklearn.metrics import mean_absolute_error
from sklearn.impute import SimpleImputer
from sklearn.cluster import KMeans

In [322]:
uploaded = files.upload()

In [323]:
# getting my category encoder
!pip install category_encoders

In [0]:
# looking at my dataframe
#data from: https://www.kaggle.com/chenzhiliang/housing-data
df = pd.read_csv('Housing_Data.csv')

In [325]:
df.shape

(24564, 11)

In [326]:
df.head()

,Unnamed: 0,districtText,furnishingCode,value,floorArea,bedrooms,bathrooms,pricePerSqFt,hasFloorplans,latitude,longitude
0,0,Hougang / Punggol / Sengkang,FULL,458000,990,3,2,462.6262626,True,1.402608,103.911427
1,1,Buona Vista / West Coast / Clementi Ne...,FULL,775000,1022,3,2,758.3170254,False,1.306192,103.782499
2,2,Ang Mo Kio / Bishan / Thomson,UNFUR,875000,1259,3,2,694.9960286,True,1.371803482,103.8535492
3,3,Admiralty / Woodlands,UNKNOWN,280000,699,3,2,400.5722461,False,1.442670315,103.7778019
4,4,Hougang / Punggol / Sengkang,PART,410000,1184,3,2,346.2837838,True,1.393140738,103.8883531


In [0]:
df['full'] = df['furnishingCode'] == 'FULL'

In [0]:
df['full'] = df['full'].astype(int)

In [0]:
df['hasFloorplans'] = df['hasFloorplans'].astype(int)

In [0]:
df['latitude'] = df['latitude'].replace({' Partially  Furnished ': np.nan, ' Fully  Furnished ': np.nan, 'Unfurnished': np.nan, 'UNKNOWN': np.nan})

In [0]:
df['latitude'] = df['latitude'].astype(float)

In [0]:
df['longitude'] = df['longitude'].replace({' Partially  Furnished ': np.nan, ' Fully  Furnished ': np.nan, 'Unfurnished': np.nan, 'UNKNOWN': np.nan})

In [0]:
df['longitude'] = df['longitude'].astype(float)

In [0]:
df['floorArea'] = df['floorArea'].replace({'sqft': np.nan })

In [335]:
df['floorArea'].astype(float)

0        990.000
1       1022.000
2       1259.000
3        699.000
4       1184.000
          ...   
24559   1119.000
24560   1593.000
24561   1388.000
24562    990.000
24563    990.000
Name: floorArea, Length: 24564, dtype: float64

In [336]:
df.dtypes

Unnamed: 0          int64
districtText       object
furnishingCode     object
value               int64
floorArea          object
bedrooms            int64
bathrooms          object
pricePerSqFt       object
hasFloorplans       int64
latitude          float64
longitude         float64
full                int64
dtype: object

In [337]:
df.head()

,Unnamed: 0,districtText,furnishingCode,value,floorArea,bedrooms,bathrooms,pricePerSqFt,hasFloorplans,latitude,longitude,full
0,0,Hougang / Punggol / Sengkang,FULL,458000,990,3,2,462.6262626,1,1.403,103.911,1
1,1,Buona Vista / West Coast / Clementi Ne...,FULL,775000,1022,3,2,758.3170254,0,1.306,103.782,1
2,2,Ang Mo Kio / Bishan / Thomson,UNFUR,875000,1259,3,2,694.9960286,1,1.372,103.854,0
3,3,Admiralty / Woodlands,UNKNOWN,280000,699,3,2,400.5722461,0,1.443,103.778,0
4,4,Hougang / Punggol / Sengkang,PART,410000,1184,3,2,346.2837838,1,1.393,103.888,0


In [0]:
df['bathrooms'].replace({'UNKNOWN': np.NaN}, inplace=True)
df['pricePerSqFt'].replace({'sqrt' : ''}, inplace=True)
df['districtText'].replace({'/':' '}, inplace=True)


In [0]:
pd.set_option('display.float_format', lambda x: '%.3f' % x)

In [340]:
df['value'].describe()

count     24564.000
mean     481441.236
std      156289.207
min      210000.000
25%      375000.000
50%      445000.000
75%      550000.000
max     2150000.000
Name: value, dtype: float64

In [0]:
df['MB'] = df['value'].mean()

In [342]:
df['MB']

0       481441.236
1       481441.236
2       481441.236
3       481441.236
4       481441.236
           ...    
24559   481441.236
24560   481441.236
24561   481441.236
24562   481441.236
24563   481441.236
Name: MB, Length: 24564, dtype: float64

In [343]:
# mmean base line error
mean_absolute_error(df['MB'], df['value'])

118831.98560338319

In [344]:
#clean this column
df['districtText'].nunique()

45

In [345]:
#catergorical encoder
df['furnishingCode'].unique()

array(['FULL', 'UNFUR', 'UNKNOWN', 'PART'], dtype=object)

In [346]:
# checking missing values
df.isnull().count()

Unnamed: 0        24564
districtText      24564
furnishingCode    24564
value             24564
floorArea         24564
bedrooms          24564
bathrooms         24564
pricePerSqFt      24564
hasFloorplans     24564
latitude          24564
longitude         24564
full              24564
MB                24564
dtype: int64

In [347]:
value_checher(df)


 0         990
1        1022
2        1259
3         699
4        1184
         ... 
24559    1119
24560    1593
24561    1388
24562     990
24563     990
Name: floorArea, Length: 24564, dtype: object 
 is missing this False    24563
True         1
Name: floorArea, dtype: int64 


 0        3
1        3
2        3
3        3
4        3
        ..
24559    3
24560    4
24561    3
24562    3
24563    3
Name: bedrooms, Length: 24564, dtype: int64 
 is missing this False    24564
Name: bedrooms, dtype: int64 


 0          2
1          2
2          2
3          2
4          2
        ... 
24559    NaN
24560    NaN
24561    NaN
24562      2
24563      2
Name: bathrooms, Length: 24564, dtype: object 
 is missing this False    23870
True       694
Name: bathrooms, dtype: int64 


 0        462.6262626
1        758.3170254
2        694.9960286
3        400.5722461
4        346.2837838
            ...     
24559    446.8275246
24560    351.5379787
24561     288.184438
24562    535.3535354
2456

,Unnamed: 0,districtText,furnishingCode,value,floorArea,bedrooms,bathrooms,pricePerSqFt,hasFloorplans,latitude,longitude,full,MB
0,0,Hougang / Punggol / Sengkang,FULL,458000,990,3,2,462.6262626,1,1.403,103.911,1,481441.236
1,1,Buona Vista / West Coast / Clementi Ne...,FULL,775000,1022,3,2,758.3170254,0,1.306,103.782,1,481441.236
2,2,Ang Mo Kio / Bishan / Thomson,UNFUR,875000,1259,3,2,694.9960286,1,1.372,103.854,0,481441.236
3,3,Admiralty / Woodlands,UNKNOWN,280000,699,3,2,400.5722461,0,1.443,103.778,0,481441.236
4,4,Hougang / Punggol / Sengkang,PART,410000,1184,3,2,346.2837838,1,1.393,103.888,0,481441.236
...,...,...,...,...,...,...,...,...,...,...,...,...,...
24559,25093,Dairy Farm / Bukit Panjang / Choa Chu ...,UNKNOWN,500000,1119,3,NaN,446.8275246,0,1.383,103.744,0,481441.236
24560,25094,Dairy Farm / Bukit Panjang / Choa Chu ...,UNKNOWN,560000,1593,4,NaN,351.5379787,0,1.393,103.741,0,481441.236
24561,25095,Dairy Farm / Bukit Panjang / Choa Chu ...,UNKNOWN,400000,1388,3,NaN,288.184438,0,1.393,103.744,0,481441.236
24562,25096,Hougang / Punggol / Sengkang,PART,530000,990,3,2,535.3535354,0,1.385,103.894,0,481441.236


In [0]:
# df = df.dropna()

In [0]:
train = df

In [0]:
target = 'value'

features = train.columns.drop([target, 'districtText', 'hasFloorplans', 'Unnamed: 0'])

In [351]:
train

,Unnamed: 0,districtText,furnishingCode,value,floorArea,bedrooms,bathrooms,pricePerSqFt,hasFloorplans,latitude,longitude,full,MB
0,0,Hougang / Punggol / Sengkang,FULL,458000,990,3,2,462.6262626,1,1.403,103.911,1,481441.236
1,1,Buona Vista / West Coast / Clementi Ne...,FULL,775000,1022,3,2,758.3170254,0,1.306,103.782,1,481441.236
2,2,Ang Mo Kio / Bishan / Thomson,UNFUR,875000,1259,3,2,694.9960286,1,1.372,103.854,0,481441.236
3,3,Admiralty / Woodlands,UNKNOWN,280000,699,3,2,400.5722461,0,1.443,103.778,0,481441.236
4,4,Hougang / Punggol / Sengkang,PART,410000,1184,3,2,346.2837838,1,1.393,103.888,0,481441.236
...,...,...,...,...,...,...,...,...,...,...,...,...,...
24559,25093,Dairy Farm / Bukit Panjang / Choa Chu ...,UNKNOWN,500000,1119,3,NaN,446.8275246,0,1.383,103.744,0,481441.236
24560,25094,Dairy Farm / Bukit Panjang / Choa Chu ...,UNKNOWN,560000,1593,4,NaN,351.5379787,0,1.393,103.741,0,481441.236
24561,25095,Dairy Farm / Bukit Panjang / Choa Chu ...,UNKNOWN,400000,1388,3,NaN,288.184438,0,1.393,103.744,0,481441.236
24562,25096,Hougang / Punggol / Sengkang,PART,530000,990,3,2,535.3535354,0,1.385,103.894,0,481441.236


In [352]:
df['bathrooms']

0          2
1          2
2          2
3          2
4          2
        ... 
24559    NaN
24560    NaN
24561    NaN
24562      2
24563      2
Name: bathrooms, Length: 24564, dtype: object

In [0]:
train, test = train_test_split(train,
                               train_size=0.80,
                               test_size=0.20,
                               random_state=42)

In [354]:
df.describe(exclude='number')

,districtText,furnishingCode,floorArea,bathrooms,pricePerSqFt
count,24564,24564,24563,23870,24564
unique,45,4,555,5,7876
top,Hougang / Punggol / Sengkang,PART,1184,2,380.0675676
freq,4925,9100,2163,21259,199


In [0]:
df['floorArea'] = df['floorArea'].astype(float)

In [0]:
df['bathrooms'] = df['bathrooms'].astype(float)

In [0]:
df['pricePerSqFt'] = df['pricePerSqFt'].replace({'sqft': np.nan})

In [0]:
df['pricePerSqFt'] = df['pricePerSqFt'].astype(float)

In [359]:
df.describe()

,Unnamed: 0,value,floorArea,bedrooms,bathrooms,pricePerSqFt,hasFloorplans,latitude,longitude,full,MB
count,24564.000,24564.000,24563.000,24564.000,23870.000,24563.000,24564.000,24117.000,24478.000,24564.000,24564.000
mean,12587.644,481441.236,1090.357,2.932,2.002,501.423,0.441,1.382,102.313,0.226,481441.236
std,7265.709,156289.207,848.994,0.604,0.341,517.167,0.497,1.152,12.395,0.418,0.000
min,0.000,210000.000,56.000,1.000,1.000,3.846,0.000,-0.000,0.000,0.000,481441.236
25%,6261.750,375000.000,914.500,3.000,2.000,369.150,0.000,1.341,103.773,0.000,481441.236
50%,12668.500,445000.000,1108.000,3.000,2.000,415.648,0.000,1.371,103.842,0.000,481441.236
75%,18888.250,550000.000,1280.000,3.000,2.000,486.726,1.000,1.398,103.890,0.000,481441.236
max,25097.000,2150000.000,124816.000,7.000,6.000,10614.525,1.000,150.000,130.000,1.000,481441.236


In [360]:
train.shape, test.shape

((19651, 13), (4913, 13))

In [0]:
train, val = train_test_split(train,
                              train_size=0.80,
                              test_size=0.20,
                              random_state=43)

In [362]:
train.shape, val.shape, test.shape 

((15720, 13), (3931, 13), (4913, 13))

In [0]:
X_train = train[features]
y_train = train[target]
X_val = val[features]
y_val = val[target]
X_test = test[features]
y_test = test[target]

In [364]:
X_train.shape, X_val.shape, X_test.shape

((15720, 9), (3931, 9), (4913, 9))

In [0]:
encoder = ce.OneHotEncoder()
# X_train_encode = encoder.fit_transform(X_train)

In [0]:
import matplotlib.pyplot as plt


In [0]:
# # Elbow Plot for k (1-10)
# K = range(1,11)
# # The average distance from each centroid, to the points in its cluster
# distortion = []
# for k in K:
#   kmeans = KMeans(n_clusters = k)
#   kmeans.fit(X_train_encode)
#   distortion.append(kmeans.inertia_)
# distortion


In [0]:
# Elbow Plot
plt.plot(K, distortion, 'bx-')
plt.title("Elbow Plot")
plt.show()

In [0]:
pipeline = make_pipeline(
    ce.OrdinalEncoder(),
    ce.OneHotEncoder(),
    # KMeans(n_clusters=15),
    SimpleImputer(strategy='most_frequent'),
    RandomForestRegressor(random_state=99, n_jobs=-1),
    )

In [373]:
pipeline.fit(X_train, y_train)

Pipeline(memory=None,
         steps=[('onehotencoder',
                 OneHotEncoder(cols=['furnishingCode', 'floorArea', 'bathrooms',
                                     'pricePerSqFt'],
                               drop_invariant=False, handle_missing='value',
                               handle_unknown='value', return_df=True,
                               use_cat_names=False, verbose=0)),
                ('simpleimputer',
                 SimpleImputer(add_indicator=False, copy=True, fill_value=None,
                               missing_values=nan, strategy='most_fre...
                 RandomForestRegressor(bootstrap=True, ccp_alpha=0.0,
                                       criterion='mse', max_depth=None,
                                       max_features='auto', max_leaf_nodes=None,
                                       max_samples=None,
                                       min_impurity_decrease=0.0,
                                       min_impurity_split=None,

In [374]:
y_pred = pipeline.predict(X_val)
mean_absolute_error(y_val, y_pred)

31009.6672068176

In [375]:
pipeline.score(X_val, y_val)

0.8376019749741053

In [376]:
df.shape

(24564, 13)

In [0]:
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

In [380]:
param_distributions = {
    'simpleimputer__strategy': ['mean', 'median'], 

}
### If you're on Colab, decrease n_iter & cv parameters
search = RandomizedSearchCV(
    pipeline, 
    param_distributions=param_distributions, 
    n_iter=100, 
    cv=5, 
    scoring='neg_mean_absolute_error', 
    verbose=10, 
    return_train_score=True, 
    n_jobs=-1
)
search.fit(X_train, y_train);

[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:  6.3min


KeyboardInterrupt: ignored

Your code starts here:

In [0]:
# Change column names: replace spaces with underscores


In [0]:
# Get Pandas Profiling Report


In [0]:
# Keep just the subset of data for the Tribeca neighborhood
# Check how many rows you have now. (Should go down from > 20k rows to 146)


In [0]:
# Q. What's the date range of these property sales in Tribeca?


In [0]:
# The Pandas Profiling Report showed that SALE_PRICE was read as strings
# Convert it to integers


In [0]:
# Q. What is the maximum SALE_PRICE in this dataset?


In [0]:
# Look at the row with the max SALE_PRICE


In [0]:
# Get value counts of TOTAL_UNITS
# Q. How many property sales were for multiple units?


In [0]:
# Keep only the single units


In [0]:
# Q. Now what is the max sales price? How many square feet does it have?


In [0]:
# Q. How often did $0 sales occur in this subset of the data?

# There's a glossary here: 
# https://www1.nyc.gov/site/finance/taxes/property-rolling-sales-data.page

# It says:
# A $0 sale indicates that there was a transfer of ownership without a 
# cash consideration. There can be a number of reasons for a $0 sale including 
# transfers of ownership from parents to children. 



In [0]:
# Look at property sales for > 5,000 square feet
# Q. What is the highest square footage you see?


In [0]:
# What are the building class categories?
# How frequently does each occur?


In [0]:
# Keep subset of rows:
# Sale price more than $0, 
# Building class category = Condos - Elevator Apartments

# Check how many rows you have now. (Should be 106 rows.)


In [0]:
# Make a Plotly Express scatter plot of GROSS_SQUARE_FEET vs SALE_PRICE


In [0]:
# Add an OLS (Ordinary Least Squares) trendline,
# to see how the outliers influence the "line of best fit"


In [0]:
# Look at sales for more than $35 million

# All are at 70 Vestry Street
# All but one have the same SALE_PRICE & SALE_DATE
# Was the SALE_PRICE for each? Or in total?
# Is this dirty data?


In [0]:
# Make a judgment call:
# Keep rows where sale price was < $35 million

# Check how many rows you have now. (Should be down to 90 rows.)


In [0]:
# Now that you've removed outliers,
# Look again at a scatter plot with OLS (Ordinary Least Squares) trendline


In [0]:
# Select these columns, then write to a csv file named tribeca.csv. Don't include the index.
